In [3]:
import wget, os, gzip, pickle, random, re, sys, importlib, tqdm, math, os, gzip, re, string

from tqdm import trange
from collections import Counter

import torch

import random
from random import choice

IMDB_URL = 'http://dlvu.github.io/data/imdb.{}.pkl.gz'
IMDB_FILE = 'imdb.{}.pkl.gz'
WP_DATA = 'https://codeberg.org/pbm/former/raw/branch/master/data/enwik8.gz'

PAD, START, END, UNK = '.pad', '.start', '.end', '.unk'

SENT = '_s'
TOY = {
    '_s': ['_s _adv', '_np _vp', '_np _vp _prep _np', '_np _vp ( _prep _np )', '_np _vp _con _s','_np _vp ( _con _s )'],
    '_adv': ['briefly', 'quickly', 'impatiently'],
    '_np': ['a _noun', 'the _noun', 'a _adj _noun', 'the _adj _noun'],
    '_prep': ['on', 'with', 'to', 'for', 'at'],
    '_con': ['while', 'but'],
    '_noun': ['mouse', 'bunny', 'cat', 'dog', 'man', 'woman', 'person', 'bear', 'koala', 'judge', 'businessman',
        'businesswoman', 'lawyer', 'teacher', 'engineer'],
    '_vp': ['walked', 'walks', 'ran', 'runs', 'goes', 'went', 'hiked'],
    '_adj': ['short', 'quick', 'busy', 'nice', 'gorgeous', 'spectacular', 'reluctant', 'systematic', 'willowy', 'engaged', 'synthetic']
}

PRINTABLE = set(ord(c) for c in (string.digits + string.ascii_letters + string.punctuation + string.whitespace))

def cas(i):
    """
    Character-as-string. Filters out the ascii codes that aren't safe to print.
    :return:
    """
    assert i >= 0 and i < 256
    return '□' if i not in PRINTABLE else str(chr(i))

def t(blist):
    return torch.tensor([int(b) for b in blist], dtype=torch.uint8)

def gen_sentence(sent=SENT, g=TOY):

    symb = '_[a-z]*'

    while True:

        match = re.search(symb, sent)
        if match is None:
            return sent

        s = match.span()
        sent = sent[:s[0]] + random.choice(g[sent[s[0]:s[1]]]) + sent[s[1]:]

def load_toy(ntrain=100_000, ntest=20_000, to_torch=True, final=False, seed=0):
    """
    Generates language from a toy grammar.
    :param ntrain:
    :param ntest:
    :param to_torch: Whether to return torch tensors (if false, returns python lists)
    :param final: Whether to return the test set or the validation set (True for test)
    :return:
    """

    random.seed(seed)

    train, test = '', ''
    while len(train) < ntrain:
        train += gen_sentence() + ' . '

    random.seed(seed if final else seed + 1)
    # -- change the seed so we get different test/val sets depending on `final`

    while len(test) < ntest:
        test += gen_sentence() + ' . '

    ctr = Counter(train + test)
    i2t = [PAD, START, END, UNK] + [t for t, _ in ctr.most_common()]
    t2i = { w : i for  i, w in enumerate(i2t)}

    train = [t2i[t] for t in train]
    test  = [t2i[t] for t in test]
    
    if to_torch:
        return (t(train), t(test)), (i2t, t2i) # Torch vectors (this takes a few seconds)

    return (train, test), (i2t, t2i)

def load_wp(fname='enwik8.gz', split=(90, 5, 5), to_torch=True, final=False):
    """
    Load the enwik8 dataset from the Hutter challenge as a list or vector of bytes.
    :param fname: Filename for the downloaded data.
    :param split: Percentages for the train/val/test split.
    :param to_torch: Whether to return torch tensors (True) or python lists (False)
    :param final: If False, returns train/val if True returns train/test with the validation
    data added to the training data.
    :return:
    """

    if not os.path.exists(fname):
        # If it doesn't exist, download it
        print('Downloading')
        wget.download(WP_DATA, out=fname)
        
    with gzip.open(fname, 'r') if fname.endswith('.gz') else open(fname, 'rb') as file:

        all = file.read()
        ctr = Counter(all)

        i2t = {token : cas(token) for token, freq in ctr.most_common()}
        t2i = {w : i for i, w in enumerate(i2t)}

        split = tuple(s/sum(split) for s in split)
        split = tuple(int(s * len(all)) for s in split)

        train, val, test = all[:split[0]], all[split[0]:split[0]+split[1]], all[split[0]+split[1]:]

        if final:
            train = train + val
            wh = test
        else:
            wh = val

        if to_torch:
            return (t(train), t(wh)), (i2t, t2i)

        return (train, wh), (i2t, t2i)


def load_xor(ntrain=25_000, ntest=25_000, seed=0):

    random.seed(seed)

    i2w = [PAD, START, END, UNK, 'true', 'false'] #
    w2i = {w : i for i, w in enumerate(i2w)}

    dataset, labels = [], []
    for _ in range(ntrain + ntest):
        sentence = [
            choice((i2w[4], i2w[5])),
            choice((i2w[4], i2w[5]))
        ]

        f1, f2 = (sentence[0] == i2w[4]), (sentence[1] == i2w[4]) # true: very/great false: not/terrible
        # -- these words are the only meaningful features
        label = 0 if f1 != f2 else 1

        dataset.append([w2i[word] for word in sentence])
        labels.append(label)

    return \
        (dataset[:ntrain], labels[:ntrain]), \
        (dataset[ntrain:], labels[ntrain:]), \
        (i2w, w2i), 2

def load_imdb_synth(ntrain=25_000, ntest=25_000, seed=0):
    """
    Synthetic IMDb dataset
    :param seed:
    :param voc:
    :return:
    """

    random.seed(seed)

    adjectives = ['classic', 'silent', 'modern', 'vintage', 'independent', 'foreign', 'animated', 'documentary',
    'epic', 'dramatic', 'romantic', 'comic', 'thrilling', 'mysterious', 'gritty', 'stylized', 'iconic', 'acclaimed',
    'popular', 'forgettable', 'unreleased', 'awardwinning', 'blockbuster', 'lowbudget', 'highbudget', 'experimental',
    'mainstream', 'cult', 'notable', 'original']
    nouns = ['movie', 'film', 'motion-picture', 'feature', 'featurette', 'picture', 'flick', 'cinema', 'screenplay',
    'blockbuster', 'talkie', 'silent', 'biopic', 'short', 'docudrama', 'documentary', 'animation', 'cartoon',
    'anime', 'telefilm', 'miniseries', 'drama', 'comedy', 'thriller', 'western', 'musical', 'noir']
    verbs = ['was', 'is', 'became', 'becomes', 'seemed', 'seems']

    i2w = [PAD, START, END, UNK, 'this', 'not', 'very', 'great','terrible'] + verbs + adjectives + nouns
    w2i = {w : i for i, w in enumerate(i2w)}

    dataset, labels = [], []
    for _ in range(ntrain + ntest):
        sentence = [
            i2w[4], # this
            choice(adjectives), # old
            choice(nouns), # movie
            choice(verbs), # was
            choice((i2w[5], i2w[6])),
            choice((i2w[7], i2w[8]))
        ]

        f1, f2 = (sentence[4] == i2w[6]), (sentence[5] == i2w[7]) # true: very/great false: not/terrible
        # -- these words are the only meaningful features
        label = 0 if f1 != f2 else 1

        dataset.append([w2i[word] for word in sentence])
        labels.append(label)

    return \
        (dataset[:ntrain], labels[:ntrain]), \
        (dataset[ntrain:], labels[ntrain:]), \
        (i2w, w2i), 2


def load_imdb(final=False, val=5000, seed=0, voc=None, char=False):

    cst = 'char' if char else 'word'

    imdb_url = IMDB_URL.format(cst)
    imdb_file = IMDB_FILE.format(cst)

    if not os.path.exists(imdb_file):
        wget.download(imdb_url)

    with gzip.open(imdb_file) as file:
        sequences, labels, i2w, w2i = pickle.load(file)

    if voc is not None and voc < len(i2w):
        nw_sequences = {}

        i2w = i2w[:voc]
        w2i = {w: i for i, w in enumerate(i2w)}

        mx, unk = voc, w2i['.unk']
        for key, seqs in sequences.items():
            nw_sequences[key] = []
            for seq in seqs:
                seq = [s if s < mx else unk for s in seq]
                nw_sequences[key].append(seq)

        sequences = nw_sequences

    if final:
        return (sequences['train'], labels['train']), (sequences['test'], labels['test']), (i2w, w2i), 2

    # Make a validation split
    random.seed(seed)

    x_train, y_train = [], []
    x_val, y_val = [], []

    val_ind = set( random.sample(range(len(sequences['train'])), k=val) )
    for i, (s, l) in enumerate(zip(sequences['train'], labels['train'])):
        if i in val_ind:
            x_val.append(s)
            y_val.append(l)
        else:
            x_train.append(s)
            y_train.append(l)

    return (x_train, y_train), \
           (x_val, y_val), \
           (i2w, w2i), 2

c:\Users\stent\OneDrive\Desktop\DeepLearning-assignments\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


# Part 1: Classification (Data Loading)

In [4]:
# Load the IMDb dataset
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)

In [ ]:
for i in range(5):
    print("Sequence", i)
    print([i2w[idx] for idx in x_train[i]], f"label: {y_train[i]}")
    print(x_train[i], f"label: {y_train[i]}")
    print()

Sequence 0
['this', 'movie', 'is', 'terrible', 'but', 'it', 'has', 'some', 'good', 'effects'] label: 1
[14, 19, 9, 379, 22, 11, 50, 52, 53, 290] label: 1

Sequence 1
['i', 'wouldn', 't', 'rent', 'this', 'one', 'even', 'on', 'dollar', 'rental', 'night'] label: 1
[13, 574, 25, 809, 14, 32, 63, 26, 2722, 2231, 312] label: 1

Sequence 2
['ming', 'the', 'merciless', 'does', 'a', 'little', 'bardwork', 'and', 'a', 'movie', 'most', 'foul'] label: 1
[10721, 4, 10956, 129, 6, 124, 88114, 5, 6, 19, 93, 4118] label: 1

Sequence 3
['long', 'boring', 'blasphemous', 'never', 'have', 'i', 'been', 'so', 'glad', 'to', 'see', 'ending', 'credits', 'roll'] label: 1
[198, 351, 17697, 116, 31, 13, 80, 40, 1240, 8, 69, 272, 883, 1749] label: 1

Sequence 4
['no', 'comment', 'stupid', 'movie', 'acting', 'average', 'or', 'worse', 'screenplay', 'no', 'sense', 'at', 'all', 'skip', 'it'] label: 1
[60, 913, 366, 19, 118, 836, 44, 431, 902, 60, 286, 35, 34, 1834, 11] label: 1



### Question 1

To-Do:
1. Select a batch of the IMDB dataset.
2. Determine the longest sequence in the batch.
3. Pad all other sequences in the batch with the pad token index (w2i[".pad"]).
4. Convert the result to a tensor of type torch.long.

In [ ]:
import torch

def prepare_batch(sequences, labels, batch_size, w2i):
    """
    Prepare batches with padding for transformer input.
    
    Args:
        sequences: List of sequences (each sequence is a list of token indices)
        labels: List of labels corresponding to sequences
        batch_size: Size of each batch
        w2i: Word to index dictionary (contains the padding token)
    
    Yields:
        Tuple of (padded_batch, labels_batch) where padded_batch is a tensor
    """
    # Get the padding token index
    pad_idx = w2i[PAD]
    
    # Process data in batches
    for i in range(0, len(sequences), batch_size):
        # Get batch slice
        batch_sequences = sequences[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]

        # Print raw sequence lengths
        print(f"Batch {i//batch_size} lengths:", [len(seq) for seq in batch_sequences])
        
        # Find max length in this batch
        max_len = max(len(seq) for seq in batch_sequences)
        print(f"Max sequence length in batch {i//batch_size}: {max_len}")
        
        # Pad sequences to max length
        padded_batch = []
        for seq in batch_sequences:
            # Pad sequence to max_len
            padded_seq = seq + [pad_idx] * (max_len - len(seq))
            padded_batch.append(padded_seq)
        
        # Convert to tensors
        batch_tensor = torch.tensor(padded_batch, dtype=torch.long)
        labels_tensor = torch.tensor(batch_labels, dtype=torch.long)
        
        yield batch_tensor, labels_tensor


# Example usage
batch_size = 32
batches = list(prepare_batch(x_train[:100], y_train[:100], batch_size, w2i)) # Using first 100 samples for demonstration

# Show first batch
first_batch, first_labels = batches[0]
print(f"The batch shape of the first batch: {first_batch.shape}")
print(f"The labels shape of the first batch: {first_labels.shape}")
print(f"Padding token index: {w2i[PAD]}")
print(f"\nThe first sequence of first batch:")
print(first_batch[0])
print(f"\nThe actual words of the first sequence of the first batch: {[i2w[idx.item()] for idx in first_batch[0]]}")

Batch 0 lengths: [10, 11, 12, 14, 15, 16, 17, 17, 19, 20, 22, 22, 22, 23, 24, 25, 25, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 30, 30, 30, 30, 30]
Max sequence length in batch 0: 30
Batch 1 lengths: [30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35]
Max sequence length in batch 1: 35
Batch 2 lengths: [35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38]
Max sequence length in batch 2: 38
Batch 3 lengths: [38, 38, 38, 38]
Max sequence length in batch 3: 38
The batch shape of the first batch: torch.Size([32, 30])
The labels shape of the first batch: torch.Size([32])
Padding token index: 0

The first sequence of first batch:
tensor([ 14,  19,   9, 379,  22,  11,  50,  52,  53, 290,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0])

The actual words of the first sequence of 

# Part 2: Classification (Baseline Model)

Baseline Model: The simplest seq2seq model; a linear layer applied to each token in the sequence separately.

### Question 2

Only define the model (embedding -> pooling -> linear layer)

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleClassifier(nn.Module):
    """
    Simple classification model with embedding layer and global pooling.

    Architecture:
    1. Embedding layer: converts token indices to embedding vectors
    2. Global pooling: pools along the time dimension
    3. Linear layer: projects to number of classes
    """

    def __init__(self, vocab_size, emb_size, num_classes, pooling="max"):
        """
        Args:
            vocab_size: Number of tokens in vocabulary
            emb_size: Size of embedding vectors (300)
            num_classes: Number of output classes
            pooling: Type of pooling ('max', 'mean', or 'sum')
        """
        super(SimpleClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.linear = nn.Linear(emb_size, num_classes)
        self.pooling = pooling

    def forward(self, x):
        """
        Forward pass.

        Args:
            x: Input tensor of shape (batch, time) with dtype=torch.long
            time = padded sequence length

        Returns:
            Output tensor of shape (batch, num_classes) with dtype=torch.float
        """
        # Step 1: Embedding layer - Converts token indices into embeddings
        # Input: (batch, time) -> Output: (batch, time, emb)
        embedded = self.embedding(x)

        # Step 2: Global pooling along time dimension (dim=1) - Reduces the entire sequence into one vector using a pooling method
        # Clarification: (batch, time, emb) -> (dim=0, dim=1, dim=2)
        # Input: (batch, time, emb) -> Output: (batch, emb)
        if self.pooling == "max":
            pooled, _ = torch.max(embedded, dim=1)  # max pooling
        elif self.pooling == "mean":
            pooled = torch.mean(embedded, dim=1)  # mean pooling
        elif self.pooling == "sum":
            pooled = torch.sum(embedded, dim=1)  # sum pooling
        
        elif self.pooling == "select":
            # Example: select the first time step
            pooled = embedded[:, 0, :]  # select pooling
        else:
            raise ValueError(f"Unknown pooling type: {self.pooling}")

        # Step 3: Classifier layer - Projects pooled vector to number of classes
        # Input: (batch, emb) -> Output: (batch, num_classes)
        output = self.linear(pooled)

        # Note: We don't apply softmax here - it's included in cross_entropy loss
        return output


# Model hyperparameters
emb_size = 300
vocab_size = len(i2w)  # Get vocabulary size from i2w
num_classes = numcls  # Number of classes (2 for binary classification)

# Create model
model = SimpleClassifier(vocab_size, emb_size, num_classes, pooling="mean")

print(f"Model created:")
print(f"  Vocabulary size: {vocab_size}")
print(f"  Embedding size: {emb_size}")
print(f"  Number of classes: {num_classes}")
print(f"\nModel architecture:")
print(model)

# Test with a small batch
test_batch, test_labels = next(
    prepare_batch(x_train, y_train, batch_size=500, w2i=w2i)
)
print(f"\nTest batch shape: {test_batch.shape}")
output = model(test_batch)
print(f"Output shape: {output.shape}")
print(f"Output dtype: {output.dtype}")

# Calculate loss using cross_entropy (includes softmax)
loss = F.cross_entropy(output, test_labels)
print(f"\nLoss: {loss.item():.4f}")

# Calculate accuracy
predictions = torch.argmax(output, dim=1)
accuracy = (predictions == test_labels).float().mean()
print(f"Accuracy: {accuracy.item():.4f}")

Model created:
  Vocabulary size: 99430
  Embedding size: 300
  Number of classes: 2

Model architecture:
SimpleClassifier(
  (embedding): Embedding(99430, 300)
  (linear): Linear(in_features=300, out_features=2, bias=True)
)
Batch 0 lengths: [10, 11, 12, 14, 15, 16, 17, 17, 19, 20, 22, 22, 22, 23, 24, 25, 25, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 

### Question 3

A continuation of Q2

Steps:
1. Train our baseline model (write a training loop) on the IMDb dataset
2. Try different global pooling methods (max, mean, sum, select)
3. Compare their performance (use a cross entropy loss and an optimizer)
4. Explain the results (track the training loss and accuracy)

**The goal is to compare how each pooling performs on sentiment classification.**

In [ ]:
import torch.optim as optim
from tqdm import tqdm

# Training hyperparameters
batch_size = 500
learning_rate = 0.003

# Initialize optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train_epoch(model, sequences, labels, batch_size, w2i, optimizer):
    """
    Train for one epoch.
    
    Args:
        model: The model to train
        sequences: Training sequences
        labels: Training labels
        batch_size: Batch size
        w2i: Word to index dictionary
        optimizer: Optimizer
        
    Returns:
        Average loss and accuracy for the epoch
    """
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    # Create batches
    num_batches = (len(sequences) + batch_size - 1) // batch_size
    
    with tqdm(total=num_batches, desc="Training", ncols=100) as pbar:
        for batch_data, batch_labels in prepare_batch(sequences, labels, batch_size, w2i):
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_data)
            
            # Calculate loss
            loss = F.cross_entropy(outputs, batch_labels)
            
            # Backward pass
            loss.backward()
            
            # Update weights
            optimizer.step()
            
            # Calculate accuracy
            predictions = torch.argmax(outputs, dim=1)
            correct = (predictions == batch_labels).sum().item()
            
            # Update statistics
            total_loss += loss.item() * batch_data.size(0)
            total_correct += correct
            total_samples += batch_data.size(0)
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{correct / batch_data.size(0):.4f}'
            })
            pbar.update(1)
    
    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples
    
    return avg_loss, avg_acc


# Validation function
def evaluate(model, sequences, labels, batch_size, w2i):
    """
    Evaluate the model on validation/test data.
    
    Args:
        model: The model to evaluate
        sequences: Validation sequences
        labels: Validation labels
        batch_size: Batch size
        w2i: Word to index dictionary
        
    Returns:
        Average loss and accuracy
    """
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch_data, batch_labels in prepare_batch(sequences, labels, batch_size, w2i):
            # Forward pass
            outputs = model(batch_data)
            
            # Calculate loss
            loss = F.cross_entropy(outputs, batch_labels)
            
            # Calculate accuracy
            predictions = torch.argmax(outputs, dim=1)
            correct = (predictions == batch_labels).sum().item()
            
            # Update statistics
            total_loss += loss.item() * batch_data.size(0)
            total_correct += correct
            total_samples += batch_data.size(0)
    
    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples
    
    return avg_loss, avg_acc


# Train for 1 epoch
print("Training for 1 epoch...")
print(f"Training samples: {len(x_train)}")
print(f"Validation samples: {len(x_val)}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}\n")

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    train_loss, train_acc = train_epoch(model, x_train, y_train, batch_size, w2i, optimizer)

print(f"\nTraining Results:")
print(f"  Loss: {train_loss:.4f}")
print(f"  Accuracy: {train_acc:.4f}")

# Evaluate on validation set
print("\nEvaluating on validation set...")
val_loss, val_acc = evaluate(model, x_val, y_val, batch_size, w2i)

print(f"\nValidation Results:")
print(f"  Loss: {val_loss:.4f}")
print(f"  Accuracy: {val_acc:.4f}")

Training for 1 epoch...
Training samples: 20000
Validation samples: 5000
Batch size: 500
Learning rate: 0.003

Epoch 1


Training:   5%|█▍                           | 2/40 [00:00<00:02, 13.53it/s, loss=0.6817, acc=0.5940]

Batch 0 lengths: [10, 11, 12, 14, 15, 16, 17, 17, 19, 20, 22, 22, 22, 23, 24, 25, 25, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45

Training:  15%|████▎                        | 6/40 [00:00<00:02, 15.09it/s, loss=0.6836, acc=0.5200]

Batch 3 lengths: [80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 84, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86

...

Batch 36 lengths: [470, 470, 470, 470, 470, 471, 471, 471, 471, 471, 471, 471, 471, 472, 472, 472, 472, 473, 473, 473, 473, 473, 473, 473, 473, 473, 473, 474, 474, 474, 474, 474, 474, 474, 474, 474, 474, 474, 474, 475, 475, 475, 475, 475, 475, 475, 475, 475, 475, 476, 476, 476, 476, 476, 476, 476, 476, 476, 476, 476, 477, 477, 477, 477, 477, 477, 477, 478, 478, 478, 478, 478, 478, 478, 478, 478, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 480, 480, 480, 480, 480, 480, 480, 480, 480, 481, 481, 481, 481, 481, 481, 481, 481, 481, 481, 481, 482, 482, 482, 482, 482, 482, 482, 482, 482, 482, 482, 482, 482, 482, 483, 483, 483, 483, 484, 484, 484, 484, 484, 484, 484, 484, 484, 484, 485, 485, 485, 485, 485, 485, 485, 485, 486, 486, 486, 486, 486, 486, 486, 486, 486, 486, 487, 487, 487, 487, 487, 487, 487, 487, 487, 487, 487, 487, 487, 488, 488, 488, 488, 488, 489, 489, 489, 489, 489, 489, 489, 489, 489, 489, 490, 490, 490, 490, 490, 490, 490, 490, 490, 490, 4

Training:  98%|███████████████████████████▎| 39/40 [00:03<00:00,  7.44it/s, loss=0.3548, acc=0.8920]

Batch 38 lengths: [615, 616, 616, 617, 617, 617, 617, 617, 617, 617, 617, 617, 618, 618, 618, 618, 618, 619, 619, 619, 619, 619, 620, 620, 620, 620, 620, 620, 620, 621, 622, 622, 622, 622, 622, 622, 622, 622, 623, 623, 623, 624, 624, 624, 624, 625, 625, 625, 625, 625, 626, 626, 626, 626, 627, 627, 627, 627, 627, 628, 628, 628, 628, 629, 629, 629, 630, 630, 630, 630, 630, 630, 630, 630, 631, 631, 631, 631, 633, 633, 633, 633, 633, 633, 633, 633, 633, 634, 634, 634, 634, 634, 635, 635, 635, 635, 635, 636, 636, 636, 636, 636, 636, 636, 637, 637, 637, 637, 638, 638, 638, 639, 639, 639, 639, 640, 640, 640, 640, 640, 641, 641, 641, 641, 641, 641, 642, 642, 642, 642, 642, 642, 642, 643, 643, 643, 643, 643, 643, 644, 644, 644, 645, 645, 645, 645, 645, 646, 646, 647, 647, 648, 648, 648, 648, 649, 649, 649, 649, 650, 650, 650, 650, 650, 651, 652, 652, 652, 652, 652, 652, 653, 653, 653, 653, 654, 654, 654, 655, 655, 656, 656, 656, 656, 656, 656, 656, 657, 657, 658, 658, 658, 659, 659, 659, 660, 6

Training: 100%|████████████████████████████| 40/40 [00:03<00:00, 10.14it/s, loss=1.2517, acc=0.4640]



Training Results:
  Loss: 0.3169
  Accuracy: 0.8903

Evaluating on validation set...
Batch 0 lengths: [12, 13, 15, 16, 19, 20, 20, 24, 24, 26, 27, 27, 28, 29, 29, 29, 30, 30, 30, 31, 32, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 60, 60, 60, 60, 60, 60, 60, 60, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 